# setup

In [227]:
#from elasticsearch_dsl import Search, Q, Range, connections
#from elasticsearch.client import Elasticsearch
from tqdm.notebook import tqdm
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import os
import json
import re
from openai import OpenAI
import numpy as np
import ast

In [135]:
EVENTS_FOLDER = "../data/events/"

In [136]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 20)
pd.set_option('display.max_colwidth', None)

In [137]:
df_events = pd.read_pickle(f"{EVENTS_FOLDER}df_events have to add meslis 2025-01-07_22-23-01.pkl")

In [138]:
# shuffle the data rows 
df_events.sample(frac=1).reset_index(drop=True)

,StartDate,StartTime,EndTime,EndDate,City,StreetHouseNumber,LocationDetails,LocationName,isOnMultipleDays,origin_id,EventCategory,Source,lat,lon,Street,HouseNumber,StartTimeIsAssumed,StartDateTime,EndDateTime,EventDescription
0,2024-09-03,09,17,2024-10-31,Dresden,Junghannsstraße 1-3,01277 Dresden Junghannsstraße 1-3,Technische Sammlungen Dresden,True,45199,Ausstellungen & Museen,browse_ai,NaN,NaN,None,None,False,NaT,NaT,NaN
1,2024-02-27,None,None,NaN,Dresden,Jüdenhof 9,None,AMEDIA Plaza Dresden,True,1501823172,Gastronomie (mit Übernachtung),Gema,51.051818,13.738774,Jüdenhof,9,True,NaT,NaT,NaN
2,2024-02-27 00:00:00,None,None,NaN,Dresden,Alaunstr. 36,None,Kulturzentrum Scheune,False,1006725804,Freizeit/Sport/Kultur,Gema,51.066292,13.751645,Alaunstr.,36,True,NaT,NaT,NaN
3,None,None,None,None,Dresden,None,"Theaterplatz 1, 01067 Dresden, Germany",Wallpavillon im Dresdner Zwinger,False,db72c880-810f-11ef-aa2c-ed8353722086,None,eventim.de,NaN,NaN,Theaterplatz,1,None,2024-10-16 17:00:00,NaT,Sichere dir jetzt deine Tickets für Vivaldi „Die vier Jahreszeiten“ - Galakonzert im Dresdner Zwinger - DRESDNER RESIDENZ ORCHESTER am 16.10.2024 17:00 - Wallpavillon im Dresdner Zwinger in DRESDEN!
4,2024-02-01 00:00:00,None,None,NaN,Dresden,Nürnberger Str. 26,None,Sultan Palast,False,1006738085,Gastronomie (ohne Übernachtung),Gema,51.034500,13.720740,Nürnberger Str.,26,True,NaT,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14384,2024-02-19,None,None,NaN,Dresden,Webergasse 1,None,Play Off American Sports Bar,True,1006781443,Gastronomie (ohne Übernachtung),Gema,51.049920,13.735641,Webergasse,1,True,NaT,NaT,NaN
14385,2024-09-22,18,NaN,NaT,Freiburg,Am Predigertor 1,NaN,Am Predigertor (Ecke Rotteckring/Unterlinden),False,10779,NaN,browse_ai,NaN,NaN,None,None,False,NaT,NaT,NaN
14386,None,None,None,None,Freiburg im breisgau,None,"Franziskanerstraße 11, 79098 Freiburg im breisgau, Germany",Betty BBQ Meeting Point,False,3c48dc90-84c0-11ef-a013-4debcd4729c8,None,eventim.de,NaN,NaN,Franziskanerstraße,11,None,2024-10-19 11:30:00,NaT,Sichere dir jetzt deine Tickets für Die Bächleputzer City-Tour am 19.10.2024 11:30 - Betty BBQ Meeting Point in FREIBURG IM BREISGAU!
14387,2024-01-05,None,None,NaN,Dresden,Kraftwerk Mitte 1,None,Staatsoperette Dresden Mitte,True,1403785994,Freizeit/Sport/Kultur,Gema,51.052397,13.722742,Kraftwerk Mitte,1,True,NaT,NaT,NaN


# meslis

In [139]:
df_meslis = pd.read_pickle(f"{EVENTS_FOLDER}meslis/df_meslis 2024-12-19_12-41-21.pkl")

In [140]:
df_meslis.isCancelled.value_counts()

isCancelled
False    3269
True       18
Name: count, dtype: int64

In [141]:
flt = df_meslis.isCancelled==True
df_meslis = df_meslis[~flt]

### standardize columns

In [142]:
df_meslis.rename(columns = {
    "id": "origin_id",
    "startDate": "StartDateTime",
    "endDate": "EndDateTime",
    "mergedLocation_name": "LocationName",
    "mergedLocation_address_city": "City",
    "mergedLocation_address_street": "Street",
    "mergedLocation_address_houseNumber": "HouseNumber",
    "mergedLocation_address_streetHouseNumber": "StreetHouseNumber",
    "mergedLocation_coordinate_lat": "lat",
    "mergedLocation_coordinate_lon": "lon", 

    
}, inplace=True)

In [143]:
df_meslis.isDeleted.value_counts()

isDeleted
False    2917
True      352
Name: count, dtype: int64

In [144]:
df_meslis = df_meslis[df_meslis.isDeleted == False]

In [145]:
# for col in df_meslis.columns:
#     if "cate" in col.lower():
#         print(col)
        

In [146]:
# df_meslis.mergedLocation_category.unique()

In [147]:
df_meslis.head()

,origin_id,oldIds,isDeleted,eventName,StartDateTime,EndDateTime,isCancelled,attendingCount,isLocationSameAsOrganizer,mergedLocation_customers,mergedLocation_id,mergedLocation_oldIds,mergedLocation_isDeleted,mergedLocation_allPlaces,mergedLocation_statistic,mergedLocation_hiddenForCustomers,mergedLocation_onlyForCustomer,LocationName,mergedLocation_address_country_shortName,mergedLocation_address_country_longName,mergedLocation_address_region,mergedLocation_address_county,City,mergedLocation_address_district,mergedLocation_address_postalCode,Street,HouseNumber,StreetHouseNumber,mergedLocation_address_coordinate,mergedLocation_address_addressDescription,mergedLocation_category,mergedLocation_commercialId,mergedLocation_telephoneNumbers,mergedLocation_emailAddresses,mergedLocation_emailAddressesAsString,mergedLocation_telephoneNumbersAsString,mergedLocation_websites,mergedLocation_websitesAsString,lat,lon,mergedLocation_score,mergedLocation_elasticsearchTimestamp,mergedLocation_elasticsearchIndexTimestamp,mergedOrganizers,mergedOrganizersCount,customers,setLists,admissionFees,admissionFeesCount,lineup,lineupCount,eventTags,eventTagsCount,rating,latestUpdatedTimestamp,eventAdvertisements,countOfEventAdvertisements,hiddenForCustomers,onlyForCustomer,score,elasticsearchTimestamp,elasticsearchIndexTimestamp,mergedLocation_coordinate,mergedLocation_address_coordinate_lat,mergedLocation_address_coordinate_lon,timestamp
0,73867682,,False,"Stadtrundgang ""Dunkle Wolken über Freiburg""",2023-06-14T17:00:00+02:00,2023-06-14T18:30:00+02:00,False,0,False,"{'name': 'GEMA', 'matchedLocationIds': [], 'eventsCountPerYear': None, 'ownerStatus': 0, 'ownerStatusAsString': 'New', 'mergedLocationStatus': None}",15757685,,False,None,None,,None,Rathausplatz Freiburg im Breisgau,None,Germany,Baden-Württemberg,None,Freiburg im Breisgau,None,79102,None,None,None,NaN,None,None,None,,,,,,,47.983300,7.850000,None,2023-08-07T11:59:11.7006571+02:00,None,"{'customers': [], 'id': 8398866, 'oldIds': [], 'isDeleted': False, 'allPlaces': None, 'statistic': None, 'hiddenForCustomers': [], 'onlyForCustomer': None, 'name': 'Dokumentationszentrum Nationalsozialismus Freiburg', 'address': None, 'category': None, 'commercialId': None, 'telephoneNumbers': [], 'emailAddresses': [], 'emailAddressesAsString': '', 'telephoneNumbersAsString': '', 'websites': ['https://www.facebook.com/100083266844354'], 'websitesAsString': 'https://www.facebook.com/100083266844354', 'coordinate': None, 'score': None, 'elasticsearchTimestamp': '2023-08-07T17:00:43.6360011+02:00', 'elasticsearchIndexTimestamp': None}",1,,,,0,,0,,0,0.0,2023-07-29T21:05:56,"{'id': 66852933, 'eventName': 'Stadtrundgang ""Dunkle Wolken über Freiburg""', 'description': 'Stadtrundgang ""Dunkle Wolken über Freiburg"" Bei einem Stadtrundgang spricht Heiko Wegmann, Historiker und Autor des Buchs „Dunkle Wolken über Freiburg“, über die Themen „Nazifizierung“ von Bibliotheken, Beschlagnahmungen und Raub. Treffpunkt ist das Rathaus in der Innenstadt. Die Veranstaltung ist Teil der Reihe „Dunkle Wolken über Freiburg“. Die Reihe ist eine Kooperation des Dokumentationszentrums Nationalsozialismus, des Stadtarchivs Freiburg, der Universitätsbibliothek Freiburg, der Stadtbibliothek Freiburg und von der Initiative Verbrannte Orte e. V. Die Teilnahme an dem Stadtrundgang ist kostenfrei.', 'url': 'http://www.facebook.com/events/634855985122713', 'manuallyAddedByCustomers': None, 'hiddenForCustomers': None}, {'id': 66854386, 'eventName': 'Stadtrundgang ""Dunkle Wolken über Freiburg""', 'description': 'Stadtrundgang ""Dunkle Wolken über Freiburg"" Bei einem Stadtrundgang spricht Heiko Wegmann, Historiker und Autor des Buchs „Dunkle Wolken über Freiburg“, über die Themen „Nazifizierung“ von Bibliotheken, Beschlagnahmungen und Raub. Treffpunkt ist das Rathaus in der Innenstadt. Die Veranstaltung ist Teil der Reihe „Dunkle Wolken über Freiburg“. Die Reihe ist eine Kooperation des Dokumentationszentrums Nationalsozialismus, des Stad

In [148]:
df_meslis['StartTimeIsAssumed']=False

In [149]:
for col in ["StartDate", "StartTime", "EndTime", "EndDate", "LocationDetails", "EventCategory"]:
    df_meslis[col] = None

In [150]:
def define_meslis_source(ad):
    if ad =="":
        return "unknown"
    for link in ["songkick.com", "partyflock","livenation", "eventbrite.com", "setlist.fm", "bandsintown.com", "wegow.com", "concertful.com", "jambase.com", "facebook.com"]:
        if link in ad:
            return link
    return "other"

In [151]:
df_meslis['eventAdvertisements'] = df_meslis['eventAdvertisements'].fillna('')
df_meslis['Source'] = df_meslis['eventAdvertisements'].apply(define_meslis_source)

In [152]:
df_meslis['Source'].value_counts()

Source
facebook.com       2201
bandsintown.com     445
songkick.com        101
wegow.com            70
setlist.fm           47
concertful.com       19
jambase.com          18
eventbrite.com       14
livenation            1
partyflock            1
Name: count, dtype: int64

In [153]:
flt = df_meslis['Source']=="unknown"
df_meslis.loc[flt]

,origin_id,oldIds,isDeleted,eventName,StartDateTime,EndDateTime,isCancelled,attendingCount,isLocationSameAsOrganizer,mergedLocation_customers,mergedLocation_id,mergedLocation_oldIds,mergedLocation_isDeleted,mergedLocation_allPlaces,mergedLocation_statistic,mergedLocation_hiddenForCustomers,mergedLocation_onlyForCustomer,LocationName,mergedLocation_address_country_shortName,mergedLocation_address_country_longName,mergedLocation_address_region,mergedLocation_address_county,City,mergedLocation_address_district,mergedLocation_address_postalCode,Street,HouseNumber,StreetHouseNumber,mergedLocation_address_coordinate,mergedLocation_address_addressDescription,mergedLocation_category,mergedLocation_commercialId,mergedLocation_telephoneNumbers,mergedLocation_emailAddresses,mergedLocation_emailAddressesAsString,mergedLocation_telephoneNumbersAsString,mergedLocation_websites,mergedLocation_websitesAsString,lat,lon,mergedLocation_score,mergedLocation_elasticsearchTimestamp,mergedLocation_elasticsearchIndexTimestamp,mergedOrganizers,mergedOrganizersCount,customers,setLists,admissionFees,admissionFeesCount,lineup,lineupCount,eventTags,eventTagsCount,rating,latestUpdatedTimestamp,eventAdvertisements,countOfEventAdvertisements,hiddenForCustomers,onlyForCustomer,score,elasticsearchTimestamp,elasticsearchIndexTimestamp,mergedLocation_coordinate,mergedLocation_address_coordinate_lat,mergedLocation_address_coordinate_lon,timestamp,StartTimeIsAssumed,StartDate,StartTime,EndTime,EndDate,LocationDetails,EventCategory,Source


In [154]:
df_meslis.columns

Index(['origin_id', 'oldIds', 'isDeleted', 'eventName', 'StartDateTime',
       'EndDateTime', 'isCancelled', 'attendingCount',
       'isLocationSameAsOrganizer', 'mergedLocation_customers',
       'mergedLocation_id', 'mergedLocation_oldIds',
       'mergedLocation_isDeleted', 'mergedLocation_allPlaces',
       'mergedLocation_statistic', 'mergedLocation_hiddenForCustomers',
       'mergedLocation_onlyForCustomer', 'LocationName',
       'mergedLocation_address_country_shortName',
       'mergedLocation_address_country_longName',
       'mergedLocation_address_region', 'mergedLocation_address_county',
       'City', 'mergedLocation_address_district',
       'mergedLocation_address_postalCode', 'Street', 'HouseNumber',
       'StreetHouseNumber', 'mergedLocation_address_coordinate',
       'mergedLocation_address_addressDescription', 'mergedLocation_category',
       'mergedLocation_commercialId', 'mergedLocation_telephoneNumbers',
       'mergedLocation_emailAddresses',
       'merge

In [155]:
df_meslis.rename(columns = {"eventAdvertisements": "EventDescription"}, inplace=True)

In [156]:
for col in df_events.columns:
    if col not in df_meslis.columns:
        print(col)

isOnMultipleDays


In [157]:
df_meslis['isOnMultipleDays']=None # TODO add this information

In [158]:
for col in df_meslis.columns:
    if col not in df_events.columns:
        # remove column from df_meslis
        df_meslis.drop(columns=[col], inplace=True, errors='ignore')
        


### date field transformations

In [159]:
# data have incorrect time zone - remove them all before transformation

In [160]:
flt = df_meslis.EndDateTime =='0001-01-01T00:00:00Z'
df_meslis.loc[flt, "EndDateTime"]=None

In [161]:
df_meslis.StartDateTime = df_meslis.StartDateTime.astype(str)
df_meslis.EndDateTime = df_meslis.EndDateTime.astype(str)

In [162]:
df_meslis.StartDateTime

0       2023-06-14T17:00:00+02:00
1             2023-07-14T00:00:00
2             2023-07-22T20:00:00
3       2023-06-17T10:00:00+02:00
4             2024-02-29T19:30:00
                  ...            
3284          2024-10-01T20:00:00
3285          2024-02-14T19:30:00
3286          2024-03-13T20:00:00
3287          2024-03-06T19:30:00
3288          2024-03-04T19:30:00
Name: StartDateTime, Length: 2917, dtype: object

In [163]:
for str_part in ["+02:00", "+01:00", "+00:00", "Z"]:
    for colname in ['StartDateTime', "EndDateTime"]:
        df_meslis[colname] = df_meslis[colname].str.replace(str_part, "", regex=False)

In [164]:
df_meslis ["StartDateTime_len"] = df_meslis.StartDateTime.apply(len)
df_meslis ["EndDateTime_len"] = df_meslis.EndDateTime.apply(lambda x: len(x) if x else 0)

In [165]:
df_meslis.StartDateTime_len.value_counts()

StartDateTime_len
19    2917
Name: count, dtype: int64

In [166]:
df_meslis.EndDateTime_len.value_counts()

EndDateTime_len
19    1771
4     1146
Name: count, dtype: int64

In [167]:
df_meslis.EndDateTime = df_meslis.EndDateTime.str.replace("None", "", regex=False)

In [168]:
# add 1 day to end date when the interval is negative 

In [169]:
df_meslis.StartDateTime = pd.to_datetime(df_meslis.StartDateTime, format='ISO8601')

In [170]:
# sorted(df_meslis.EndDateTime.dropna().unique().tolist())

In [171]:
df_meslis.EndDateTime = pd.to_datetime(df_meslis.EndDateTime, format='ISO8601', yearfirst=True)

In [172]:
df_meslis.drop(columns=["StartDateTime_len", "EndDateTime_len"], inplace=True, errors="ignore")

In [173]:

df_meslis['Duration'] =  df_meslis.EndDateTime - df_meslis.StartDateTime

In [174]:
# sorted(df_meslis.Duration.dropna().unique().tolist())

In [175]:
df_meslis.EndDateTime.dt.hour

0       18.0
1        0.0
2        0.0
3       18.0
4       21.0
        ... 
3284     NaN
3285     0.0
3286     0.0
3287     0.0
3288     0.0
Name: EndDateTime, Length: 2917, dtype: float64

In [176]:
# if interval is negative, add 1 day to end date
flt = (df_meslis.Duration.dt.days < 0) 
df_meslis.loc[flt, "EndDateTime"] = df_meslis.loc[flt].EndDateTime + pd.Timedelta(days=1)


In [177]:
df_meslis['Duration'] =  df_meslis.EndDateTime - df_meslis.StartDateTime

In [178]:
flt = df_meslis.StartDateTime == df_meslis.EndDateTime
df_meslis.loc[flt, "EndDateTime"] = None

In [179]:
df_meslis['Duration'] =  df_meslis.EndDateTime - df_meslis.StartDateTime

In [180]:
df_meslis['Duration'].value_counts().sort_index()

Duration
0 days 00:15:00     11
0 days 00:30:00      9
0 days 00:40:00      1
0 days 00:45:00     25
0 days 01:00:00     84
                    ..
11 days 10:00:00     1
12 days 00:00:00     1
13 days 06:00:00     2
13 days 07:00:00     1
13 days 08:00:00     1
Name: count, Length: 102, dtype: int64

In [181]:
df_meslis['isOnMultipleDays']=False


In [182]:
# filter duration > 1 days and 0 hours
flt = df_meslis.Duration > pd.Timedelta(days=1)
df_meslis.loc[flt, "isOnMultipleDays"] = True


In [183]:
flt = df_meslis.isOnMultipleDays==False
df_meslis_to_add = df_meslis[flt].copy()

In [184]:
df_meslis_left = df_meslis[~flt]

In [185]:
flt = df_meslis_left.StartDateTime.dt.hour < df_meslis_left.EndDateTime.dt.hour 
print(len(df_meslis_left))
print(len(df_meslis_left[flt]))

78
63


In [186]:
df_events.columns

Index(['StartDate', 'StartTime', 'EndTime', 'EndDate', 'City',
       'StreetHouseNumber', 'LocationDetails', 'LocationName',
       'isOnMultipleDays', 'origin_id', 'EventCategory', 'Source', 'lat',
       'lon', 'Street', 'HouseNumber', 'StartTimeIsAssumed', 'StartDateTime',
       'EndDateTime', 'EventDescription'],
      dtype='object')

In [187]:
rows_to_append = []
flt = df_meslis_left.StartDateTime.dt.hour < df_meslis_left.EndDateTime.dt.hour 
for i, row in df_meslis_left[flt].iterrows():
    start_date_time = row["StartDateTime"]
    end_date_time = row["EndDateTime"]
    print(f"{start_date_time=}")
    print(f"{end_date_time=}")
    date_range = pd.date_range(start_date_time.date(), end_date_time.date(), freq='D')
    print(f"{date_range=}")
    
    for date in date_range: 
        row_to_append = row.copy()
        row_to_append["StartDate"] = date.strftime('%Y-%m-%d')
        row_to_append["StartDateTime"] = None
        row_to_append["EndDateTime"] = None
        #hour and minute format
        row_to_append["StartTime"] = start_date_time.strftime('%H:%M')
        row_to_append["EndTime"] = end_date_time.strftime('%H:%M')

        rows_to_append.append(row_to_append)

start_date_time=Timestamp('2023-06-24 12:00:00')
end_date_time=Timestamp('2023-06-25 13:00:00')
date_range=DatetimeIndex(['2023-06-24', '2023-06-25'], dtype='datetime64[ns]', freq='D')
start_date_time=Timestamp('2023-07-27 01:00:00')
end_date_time=Timestamp('2023-07-30 05:00:00')
date_range=DatetimeIndex(['2023-07-27', '2023-07-28', '2023-07-29', '2023-07-30'], dtype='datetime64[ns]', freq='D')
start_date_time=Timestamp('2023-07-21 09:30:00')
end_date_time=Timestamp('2023-07-23 13:00:00')
date_range=DatetimeIndex(['2023-07-21', '2023-07-22', '2023-07-23'], dtype='datetime64[ns]', freq='D')
start_date_time=Timestamp('2023-07-10 08:30:00')
end_date_time=Timestamp('2023-07-12 20:30:00')
date_range=DatetimeIndex(['2023-07-10', '2023-07-11', '2023-07-12'], dtype='datetime64[ns]', freq='D')
start_date_time=Timestamp('2024-10-30 10:00:00')
end_date_time=Timestamp('2024-11-03 16:00:00')
date_range=DatetimeIndex(['2024-10-30', '2024-10-31', '2024-11-01', '2024-11-02',
               '2024-11-03

In [189]:
len(rows_to_append)

267

In [190]:
df_meslis_left = df_meslis_left[~flt]

In [191]:
rows_to_append_2 = []
for i, row in df_meslis_left.sort_values("Duration").iterrows():
    start_date_time = row["StartDateTime"]
    end_date_time = row["EndDateTime"]
    # print(f"{start_date_time=}")
    # print(f"{end_date_time=}")
    date_range = pd.date_range(start_date_time.date(), end_date_time.date(), freq='D')
    print(f"{date_range=}")
    
    for date in date_range: 
        row_to_append = row.copy()
        row_to_append["StartDate"] = date.strftime('%Y-%m-%d')
        if date.date() == start_date_time.date():
            row_to_append["StartTime"] = start_date_time.strftime('%H:%M')
        else:
            row_to_append["StartTime"] = "00:00"

        if date.date() == end_date_time.date(): 
            row_to_append["EndTime"] = end_date_time.strftime('%H:%M')
        else:
            row_to_append["EndTime"] = "23:59"

        rows_to_append_2.append(row_to_append)
    
    print(rows_to_append_2)
    break

date_range=DatetimeIndex(['2023-06-20', '2023-06-21', '2023-06-22'], dtype='datetime64[ns]', freq='D')
[origin_id                       89847644
StartDateTime        2023-06-20 12:00:00
EndDateTime          2023-06-22 05:00:00
LocationName                  Sick Arena
City                Freiburg im Breisgau
                            ...         
LocationDetails                     None
EventCategory                       None
Source                      facebook.com
isOnMultipleDays                    True
Duration                 1 days 17:00:00
Name: 199, Length: 21, dtype: object, origin_id                       89847644
StartDateTime        2023-06-20 12:00:00
EndDateTime          2023-06-22 05:00:00
LocationName                  Sick Arena
City                Freiburg im Breisgau
                            ...         
LocationDetails                     None
EventCategory                       None
Source                      facebook.com
isOnMultipleDays                    Tr

In [193]:
# pd.DataFrame(rows_to_append_2)

In [194]:
df_events = pd.concat([df_events, df_meslis_to_add, pd.DataFrame(rows_to_append), pd.DataFrame(rows_to_append_2)], ignore_index=True)

/var/folders/zz/b00th7c11zqbbg9m2fhn24d40000gp/T/ipykernel_9608/2113738421.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_events = pd.concat([df_events, df_meslis_to_add, pd.DataFrame(rows_to_append), pd.DataFrame(rows_to_append_2)], ignore_index=True)
/var/folders/zz/b00th7c11zqbbg9m2fhn24d40000gp/T/ipykernel_9608/2113738421.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_events = pd.concat([df_events, df_meslis_to_add, pd.DataFrame(rows_to_append), pd.DataFrame(rows_to_append_2)

In [196]:
df_events

,StartDate,StartTime,EndTime,EndDate,City,StreetHouseNumber,LocationDetails,LocationName,isOnMultipleDays,origin_id,EventCategory,Source,lat,lon,Street,HouseNumber,StartTimeIsAssumed,StartDateTime,EndDateTime,EventDescription,Duration
0,2024-09-14,15,NaN,NaT,Dresden,NaN,NaN,Hauptstraße,False,5522,Verschiedenes,browse_ai,NaN,NaN,None,None,False,NaT,NaT,NaN,NaT
1,2024-09-15,10,10,NaT,Dresden,NaN,01307 Dresden Holbeinstraße 54,Sportplatz am Spielplatz Holbeinstraße,False,6046,Sportveranstaltungen,browse_ai,NaN,NaN,None,None,False,NaT,NaT,NaN,NaT
2,2024-09-21,10,19,NaT,Dresden,NaN,NaN,Zentralbibliothek,False,9873,Verschiedenes,browse_ai,NaN,NaN,None,None,False,NaT,NaT,NaN,NaT
3,2024-09-29,11,17,NaT,Dresden,NaN,NaN,Fährgarten Johannstadt,False,14288,"Feste, Märkte & Messen",browse_ai,NaN,NaN,None,None,False,NaT,NaT,NaN,NaT
4,2024-10-20,8,16,NaT,Dresden,NaN,NaN,Alberthafen,False,25938,"Feste, Märkte & Messen",browse_ai,NaN,NaN,None,None,False,NaT,NaT,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17493,2024-02-15,11:00,17:30,None,Dresden,Karl-Laux-Straße 5,None,SPIKE,True,78291804,None,facebook.com,51.013570,13.770790,Karl-Laux-Straße 5,None,False,NaT,NaT,"{'id': 71100909, 'eventName': 'Urban Art Woche Winter 2024', 'description': 'Urban Art Woche Winter 2024\r\nHier kannst du alles über Urban Art, Street-Art und Graffiti erfahren.\n\nIhr lernt die Geschichte dieser Kultur kennen und könnt euch in unsere Bücher vertiefen und inspirieren lassen. Gemeinsam erarbeiten wir Motive und stehen euch bei der Umsetzung zur Seite. Von Papieren über Lackmarker, Sprühdosen, Sticker und Leinwänden ist alles dabei. Wir zeichnen Skizzen, üben Styles und Character, malen Bilder, sprühen an unserem Legal Plain, schneiden Stencils und kleben Paste-Ups. Freut Euch auf eine spannende Woche und viele neue Skills!\n\nDer Workshop findet an jedem Tag der Workshopwoche statt. Außerdem sind Eltern und Freunde herzlich eingeladen, am letzten Tag der Workshopwoche zur Präsentation der Endergebnisse vorbeizuschauen!\n\n\nMo – Do: 11:00 – 14:00 Uhr Workshop\nFr: 13:00 – 17:00 Uhr Workshop +\n17:00 – 17:30 Uhr Präsentation\n\n\nTeilnehmerbeitrag:\n\n25€ | mit Dresden-Pass kostenfrei\n(damit kann der Workshop an jedem Tag besucht werden)\n\nAnmeldung unter:\nanmeldung@spikedresden.de \n\nWir freuen uns auf euch! ', 'url': 'http://www.facebook.com/events/1337942360434919', 'manuallyAddedByCustomers': None, 'hiddenForCustomers': None}",4 days 06:30:00
17494,2024-02-16,11:00,17:30,None,Dresden,Karl-Laux-Straße 5,None,SPIKE,True,78291804,None,facebook.com,51.013570,13.770790,Karl-Laux-Straße 5,None,False,NaT,NaT,"{'id': 71100909, 'eventName': 'Urban Art Woche Winter 2024', 'description': 'Urban Art Woche Winter 2024\r\nHier kannst du alles über Urban Art, Street-Art und Graffiti erfahren.\n\nIhr lernt die Geschichte dieser Kultur kennen und könnt euch in unsere Bücher vertiefen und inspirieren lassen. Gemeinsam erarbeiten wir Motive und stehen euch bei der Umsetzung zur Seite. Von Papieren über Lackmarker, Sprühdosen, Sticker und Leinwänden ist alles dabei. Wir zeichnen Skizzen, üben Styles und Character, malen Bilder, sprühen an unserem Legal Plain, schneiden Stencils und kleben Paste-Ups. Freut Euch auf eine spannende Woche und viele neue Skills!\n\nDer Workshop findet an jedem Tag der Workshopwoche statt. Außerdem sind Eltern und Freunde herzlich eingeladen, am letzten Tag der Workshopwoche zur Präsentation der Endergebnisse vorbeizuschauen!\n\n\nMo – Do: 11:00 – 14:00 Uhr Workshop\nFr: 13:00 – 17:00 Uhr Workshop +\n17:00 – 17:30 Uhr Präsentation\n\n\nTeilnehmerbeitrag:\n\n25€ | mit Dresden-Pass kostenfrei\n(damit kann der Workshop an jedem Tag besucht werden)\n\nAnmeldung unter:\nanmeldung@spikedresden.de \n\nWir freuen uns auf euch! ', 'url': 'http://www.facebook.com/events/1337942360434919', 'manuallyAddedByCustomers': None, 'hiddenForCustomers': None}",4 days 06:30:00
17495,2023-06-20,12:00,23:59,None,Freiburg im

In [ ]:
# flt = df_meslis.Duration.isna()
# df_meslis.loc[~flt].sort_values("Duration").tail(20)


### remove matched meslis events

In [263]:
df_meslis_original = pd.read_pickle(f"{EVENTS_FOLDER}meslis/df_meslis 2024-12-19_12-41-21.pkl")

In [264]:
len(df_meslis_original)

3287

In [265]:
for id in df_meslis_original.id.tail(30):
    print(id,end = " ")

98743707 98422106 75714126 98374314 86474279 78336434 98456754 81413557 81141496 99107780 99063409 85864975 78291804 88488375 98281617 98293929 98338837 98342020 83494084 98374278 98376733 97659225 97627374 97120141 94065953 80926592 83705817 85934654 84388211 85089543 

In [266]:
flt = df_events.origin_id.astype(str).str.contains("1006873109")
df_events[flt]

,StartDate,StartTime,EndTime,EndDate,City,StreetHouseNumber,LocationDetails,LocationName,isOnMultipleDays,origin_id,EventCategory,Source,lat,lon,Street,HouseNumber,StartTimeIsAssumed,StartDateTime,EndDateTime,EventDescription,Duration
5535,2024-03-05 00:00:00,None,None,NaN,Dresden,Schloßstr. 2,None,Kabarett Die Herkuleskeule,False,1006873109,Gastronomie (ohne Übernachtung),Gema,51.051018,13.737638,Schloßstr.,2,True,NaT,NaT,NaN,NaT


In [267]:
df_meslis_original.mergedLocation_customers.value_counts()

mergedLocation_customers
{'name': 'GEMA', 'matchedLocationIds': [], 'eventsCountPerYear': None, 'ownerStatus': 0, 'ownerStatusAsString': 'New', 'mergedLocationStatus': None}                                              2371
{'name': 'GEMA', 'matchedLocationIds': ['4001162577'], 'eventsCountPerYear': None, 'ownerStatus': 2, 'ownerStatusAsString': 'Known', 'mergedLocationStatus': None}                                  87
{'name': 'GEMA', 'matchedLocationIds': ['4000345054'], 'eventsCountPerYear': None, 'ownerStatus': 2, 'ownerStatusAsString': 'Known', 'mergedLocationStatus': None}                                  55
{'name': 'GEMA', 'matchedLocationIds': ['4000623282'], 'eventsCountPerYear': None, 'ownerStatus': 1, 'ownerStatusAsString': 'Changed', 'mergedLocationStatus': None}                                52
{'name': 'GEMA', 'matchedLocationIds': ['4001056530'], 'eventsCountPerYear': None, 'ownerStatus': 2, 'ownerStatusAsString': 'Known', 'mergedLocationStatus': None}                 

In [268]:
df_subset = df_meslis_original.loc[:,['id', "customers"]].copy()

In [269]:
df_subset.dropna(subset="customers", inplace=True)

In [270]:
flt = df_subset.customers==""
df_subset= df_subset[~flt]

In [271]:
df_subset.customers = df_subset.customers.apply(lambda x: ast.literal_eval(x))

In [272]:
df_subset['customer_len'] = df_subset.customers.apply(len)

In [273]:
df_subset.customer_len.value_counts()

customer_len
12    251
2       9
Name: count, dtype: int64

In [274]:
flt = df_subset.customer_len==2
df_subset.loc[flt, "customer_1"] = df_subset.loc[flt].customers.apply(lambda x: x[0])
df_subset.loc[flt, "customer_2"] = df_subset.loc[flt].customers.apply(lambda x: x[0])

In [275]:
df_subset.loc[df_subset.customer_len==2, "customers"]=None

In [276]:
gema_matched_ids_to_exclude = []

In [277]:
gema_matched_ids_to_exclude.extend(df_subset.customers.dropna().apply(lambda dct: dct.get("matchedEventIds")).dropna().tolist())

In [278]:
gema_matched_ids_to_exclude.extend(df_subset.customer_1.dropna().apply(lambda dct: dct.get("matchedEventIds")).dropna().tolist())

In [279]:
gema_matched_ids_to_exclude.extend(df_subset.customer_2.dropna().apply(lambda dct: dct.get("matchedEventIds")).dropna().tolist())

In [280]:
gema_matched_ids_to_exclude = [el for el in gema_matched_ids_to_exclude if el!=[]]

In [282]:
# flatten a list of lists into a single list
gema_matched_ids_to_exclude = [el for sublist in gema_matched_ids_to_exclude for el in sublist]

In [284]:
len(gema_matched_ids_to_exclude)

614

In [287]:
gema_matched_ids_to_exclude=[str(el) for el in gema_matched_ids_to_exclude]

In [288]:
len(df_events)

17498

In [292]:
flt = df_events.origin_id.astype(str).isin(gema_matched_ids_to_exclude)
print(len(df_events[flt]))
df_events= df_events[~flt]
print(len(df_events))

0
17427


In [52]:
df_events.to_pickle(f"{EVENTS_FOLDER}/df_events {datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.pkl")